# Первое знакомство с python-chess

Ща посмотрим, что за зверь этот python-chess

In [1]:
import chess.pgn

with open('tmp_data/example_data.pgn', 'r') as pgn_file:
    i = 0

    game = chess.pgn.read_game(pgn_file)
    while game and i < 3:
        print(game.headers["White"], "vs", game.headers["Black"])
        print(game.board())
        game = chess.pgn.read_game(pgn_file)

        i += 1

KACHAL vs justplaybi
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
mustroll vs pelao
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
luciano2000 vs amnezia
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


Удобно!

# Эвристики интересности хода

Самый простой и прямолинейный подход - придумать самим некоторые эвристики, по которым можно будет оценивать интересность хода. Давайте Приведу список эвристик и описание, почему они могут быть интересными.

Для подсчёта эвристик нам очень пригодится оценка позиции движком, поэтому сначала настроим Stockfish

1. Установите движок Stockfish с сайта https://stockfishchess.org/download/

2. Переименуйте папку в `stockfish`, и укажите путь до исполняемого файла:

In [1]:
path_to_stockfish = '../stockfish/stockfish-ubuntu-x86-64-sse41-popcnt'

Проверим, как он работает:

In [3]:
import chess
import chess.engine
import chess.pgn

engine = chess.engine.SimpleEngine.popen_uci(path_to_stockfish)

with open('tmp_data/example_data.pgn', 'r') as pgn_file:
    example_game = chess.pgn.read_game(pgn_file)
    for _ in range(5):
        example_game = chess.pgn.read_game(pgn_file)
    print(example_game.headers['Site'])

    all_moves = list(example_game.mainline_moves())

    # Создаём пустую доску (начальная позиция)
    board = chess.Board()

    # Применяем все ходы до середины партии
    for move in all_moves[:-20]:
        board.push(move)

info = engine.analyse(board, chess.engine.Limit(depth=20), multipv=5)

info[0]

https://lichess.org/zdIVv4Fu


{'string': 'NNUE evaluation using nn-37f18f62d772.nnue (6MiB, (22528, 128, 15, 32, 1))',
 'depth': 20,
 'seldepth': 24,
 'multipv': 1,
 'score': PovScore(Cp(-643), BLACK),
 'nodes': 1121908,
 'nps': 1453248,
 'hashfull': 395,
 'tbhits': 0,
 'time': 0.772,
 'pv': [Move.from_uci('d5b4'),
  Move.from_uci('d3b1'),
  Move.from_uci('c6c5'),
  Move.from_uci('c1c5'),
  Move.from_uci('e8b5'),
  Move.from_uci('f1d1'),
  Move.from_uci('b4c6'),
  Move.from_uci('d4d5'),
  Move.from_uci('e6d5'),
  Move.from_uci('e4d5'),
  Move.from_uci('g8g7'),
  Move.from_uci('b1f5'),
  Move.from_uci('c6d4'),
  Move.from_uci('d2d4')]}

Мы получили разную информацию о позиции. Позиция оценивалась с точки зрения чёрных, потому что именно они должны делать ход. Давайте разберёмся, что означает эта информация:

1. `string`: 
   - Текстовая строка с информацией о движке и используемой модели NNUE. Это полезно для проверки версии движка и его компонентов.

2. `depth`:
   - Глубина поиска. В данном случае движок анализировал позицию на 20 полуходов вперёд.

3. `seldepth`:
   - Максимальная глубина поиска, на которую движок "заглядывал" в конкретной ветке. Иногда эта глубина может превышать указанную `depth`.

4. `multipv`:
   - Порядковый номер варианта игры.

5. `score`:
   - Оценка позиции. Поле содержит объект `PovScore` с информацией о преимуществах одной из сторон:
     - `Cp`: Оценка в единицах "сентов" пешки. Здесь `-399` означает, что чёрные имеют преимущество примерно на 4 пешки.
     - `Mate`: Если позиция ведёт к мату, будет указано количество ходов до него (например, `Mate(3)`).

6. `nodes`:
   - Количество рассмотренных позиций (узлов дерева).

7. `nps`:
   - Количество узлов, обрабатываемых движком в секунду.

8. `hashfull`:
   - Заполненность хеш-таблицы движка в процентах (здесь 92%). Если значение близко к 100%, рекомендуется увеличить размер хеша для повышения производительности.

9. `tbhits`:
   - Количество обращений к базе эндшпилей (tablebase). Если база эндшпилей не используется, значение будет равно `0`.

10. `time`:
    - Время, затраченное на анализ, в секундах.

11. `pv`:
    - "Principal Variation" — основной вариант, который движок считает лучшим. Здесь это массив ходов в формате UCI (`Move.from_uci`).

Давайте теперь на основе этих данных попробуем описать метрики интересности.

Пытаясь обобщить интересность хода, можно привести следующие неформальные признаки:

1. Сложность принятия решений: Когда игрок сталкивается с выбором между несколькими сильными ходами, особенно если они ведут к различным типам позиций (например, атакующим или защищающим), это создает напряжение и интерес.

2. Тактические моменты: Например, когда появляется возможность сделать комбинацию, поставить мат или выиграть материал — такие моменты заставляют игрока искать скрытые угрозы и возможности.

3. Переломные моменты: Например, если один игрок делает ошибку, и эта ошибка немедленно меняет ход партии. Это может быть решающее изменение в позиции, которое полностью меняет баланс сил.

4. Креативность ходов: Некоторые ходы могут быть необычными или нестандартными, что добавляет интереса, так как они показывают нестандартное мышление или глубокое понимание игры.

5. Инициатива и атака: Когда один из игроков начинает доминировать на доске, используя агрессивную стратегию, это делает игру более захватывающей.

6. Психологический фактор: Когда ситуация на доске начинает тянуться к эндшпилю, где любой ход может быть решающим, и напряжение возрастает, это тоже добавляет интереса к игре.

Но такие признаки нельзя вычислить. Зато на их основе можно составить частные случаи:


### **1. Изменение оценки позиции**
Этот признак показывает, насколько сильно ход повлиял на баланс сил. Он вычисляется как разница между оценкой позиции до и после хода. Оценка позиции берется из анализа движка Stockfish. Чем больше абсолютное значение изменения, тем сильнее ход влияет на позицию.

**Вычисление**:  
```python
delta_score = -get_score(infos[i]) - get_score(infos[i - 1])
```

---

### **2. Отклонение от Principal Variation**
Этот признак показывает, насколько текущий ход отличается от рекомендованного движком варианта (первого хода в PV). Если текущий ход не совпадает с первым ходом в PV, это может указывать на креативность или ошибку.

**Вычисление**:  
```python
deviation_from_pv = 1 if move not in get_best_moves(infos[i - 1]) else 0
```

---

### **3. Угрозы противнику**
Этот признак оценивает количество угроз, создаваемых противнику после хода. Угроза считается, если фигура противника атакована хотя бы одной фигурой игрока.

**Вычисление**:  
```python
for square in chess.SQUARES:
    piece = board.piece_at(square)
    if piece and piece.color == opponent_color:
        if board.is_attacked_by(player_color, square):
            threats_to_opponent += 1
```

---

### **4. Угрозы от противника**
Этот признак оценивает количество угроз, исходящих от противника после хода. Угроза считается, если фигура игрока атакована хотя бы одной фигурой противника.

**Вычисление**:  
```python
for square in chess.SQUARES:
    piece = board.piece_at(square)
    if piece and piece.color == player_color:
        if board.is_attacked_by(opponent_color, square):
            threats_from_opponent += 1
```

---

### **5. Изменение количества легальных ходов у противника**
Этот признак показывает, насколько сильно ход ограничивает возможности противника. Вычисляется как разница между количеством легальных ходов противника до и после хода.

**Вычисление**:  
```python
legal_moves_diff = legal_moves_after - legal_moves_before
```

---

### **6. Жертва материала**
Этот признак показывает, включает ли ход жертву материала (например, пешки или фигуры). Жертва материала вычисляется как разница в стоимости материала игрока до и после хода.

**Вычисление**:  
```python
material_before = count_material(board, cur_turn)
material_after = count_material(board, cur_turn)
sacrifice = material_after - material_before
```

---

### **7. Смена преимущества**
Этот признак показывает, меняет ли ход преимущество с одной стороны на другую. Преимущество меняется, если знак оценки позиции до и после хода различен.

**Вычисление**:  
```python
advantage_change = 1 if get_score(infos[i - 1]) * get_score(infos[i]) > 0 else 0
```

---

### **8. Близость к мату**
Этот признак показывает, насколько близка позиция к мату. Если движок видит мат в Principal Variation, вычисляется средняя дистанция до мата.

**Вычисление**:  
```python
mate_distance = -get_mate_distance(infos[i])
```

---

### **9. Уязвимость короля противника**
Этот признак показывает, насколько уязвим король противника после хода. Король считается уязвимым, если он находится под атакой хотя бы одной фигурой игрока.

**Вычисление**:  
```python
king_square = board.king(board.turn)
king_under_attack = 1 if board.is_attacked_by(not board.turn, king_square) else 0
```

---

### **10. Тактические угрозы и приёмы**
Этот признак проверяет выполнение тактических приёмов, таких как взятие фигуры, шах, мат, превращение пешки или рокировка.

- **Взятие фигуры**: Проверяется, является ли ход взятием.
- **Шах**: Проверяется, ставит ли ход противника в шах.
- **Мат**: Проверяется, ставит ли ход мат.
- **Превращение пешки**: Проверяется, используется ли превращение пешки в данном ходе.
- **Рокировка**: Проверяется, является ли ход рокировкой.

**Вычисление**:  
```python
is_capture = board.is_capture(move)
is_check = board.is_check()
is_checkmate = board.is_checkmate()
cnt_promotion = sum(1 for mv in board.legal_moves if mv.promotion) // 4
is_used_promotion = move.promotion is not None
is_castling = board.is_castling(move)
```

---

### **Итоговый список признаков**
1. Изменение оценки позиции (`delta_score`).
2. Отклонение от Principal Variation (`deviation_from_pv`).
3. Угрозы противнику (`threats_to_opponent`).
4. Угрозы от противника (`threats_from_opponent`).
5. Изменение количества легальных ходов у противника (`legal_moves_diff`).
6. Жертва материала (`sacrifice`).
7. Смена преимущества (`advantage_change`).
8. Близость к мату (`mate_distance`).
9. Уязвимость короля противника (`king_under_attack`).
10. Тактические угрозы и приёмы:
   - Взятие фигуры (`is_capture`).
   - Шах (`is_check`).
   - Мат (`is_checkmate`).
   - Количество возможных превращений пешки (`cnt_promotion`).
   - Использование превращения пешки (`is_used_promotion`).
   - Рокировка (`is_castling`).

И собственно реализация вычисления этих признаков:

In [18]:
import chess
import chess.engine
import chess.pgn
import pandas as pd
from typing import List

def count_material(board, side):
    material_values = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9}
    material_count = 0
    for piece in board.piece_map().values():
        if piece.color == side:
            material_count += material_values.get(piece.piece_type, 0)
    return material_count

# Функция для вычисления дистанции до мата
def get_mate_distance(info: List[chess.engine.InfoDict]):
    distances = []
    for i in info:
        if i['score'].relative.is_mate():
            distances.append(i['score'].relative.mate())
    if len(distances) == 0:
        return 0
    return sum(distances) / len(distances)

# Функция для вычисления средней оценки
def get_score(info: List[chess.engine.InfoDict]):
    scores = []
    for i in info:
        scores.append(i['score'].relative.score(mate_score=10_000))
    return sum(scores) / len(scores)

# Функция для получения списка лучших ходов
def get_best_moves(info: List[chess.engine.InfoDict]):
    return [i['pv'][0] for i in info]

def analyze_game(game: chess.pgn.Game, engine: chess.engine.SimpleEngine, output_csv_path: str, analyze_detph: int = 20):
    data = {
        'delta_score': [],
        'deviation_from_pv': [],
        'threats_to_opponent': [],
        'threats_from_opponent': [],
        'legal_moves_diff': [],
        'sacrifice': [],
        'advantage_change': [],
        'mate_distance': [],
        'king_under_attack': [],

        'is_capture': [],
        'is_check': [],
        'is_checkmate': [],
        'cnt_promotion': [],
        'is_used_promotion': [],
        'is_castling': []
    }

    # Анализируем позицию после каждого хода
    board = chess.Board()
    infos = [engine.analyse(board, limit=chess.engine.Limit(depth=analyze_detph), multipv=3)]
    for move in game.mainline_moves():
        board.push(move)
        info = engine.analyse(board, limit=chess.engine.Limit(depth=analyze_detph), multipv=3)
        infos.append(info)
    
    # Вычисляем сами признаки
    board = chess.Board()
    for i, move in enumerate(game.mainline_moves(), start=1):
        # 1. Изменение оценки позции
        delta_score = -get_score(infos[i]) - get_score(infos[i - 1])
        data['delta_score'].append(delta_score)

        # 2. Отклонение от Principal Variation
        deviation_from_pv = 1 if move not in get_best_moves(infos[i - 1]) else 0
        data['deviation_from_pv'].append(deviation_from_pv)

        # 3-4. Угрозы противнику и угрозы от противника
        player_color = board.turn
        opponent_color = not board.turn
        threats_to_opponent = 0
        threats_from_opponent = 0

        board.push(move)
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece and piece.color == opponent_color:
                if board.is_attacked_by(player_color, square):
                    threats_to_opponent += 1
            if piece and piece.color == player_color:
                if board.is_attacked_by(opponent_color, square):
                    threats_from_opponent += 1
        board.pop()

        data['threats_to_opponent'].append(threats_to_opponent)
        data['threats_from_opponent'].append(threats_from_opponent)

        # 5. Изменение количества легальных ходов у противника
        board.turn = not board.turn # Меняем сторону, чтобы посмотреть, какие ходы он впринципе мог бы сейчас сделать
        legal_moves_before = len(list(board.legal_moves))
        board.turn = not board.turn
        board.push(move)
        legal_moves_after = len(list(board.legal_moves))
        board.pop()

        legal_moves_diff = legal_moves_after - legal_moves_before
        data['legal_moves_diff'].append(legal_moves_diff)

        # 6. Жертва материала
        if 'pv' in infos[i]:
            cur_turn = board.turn
            material_before = count_material(board, cur_turn)
            board.push(move)
            best_move_from_opponent = get_best_moves(infos[i])[0]
            board.push(best_move_from_opponent)
            material_after = count_material(board, cur_turn)
            board.pop()
            board.pop()

            data['sacrifice'] = material_after - material_before
        else:
            data['sacrifice'] = 0

        # 7. Смена преимущества
        data['advantage_change'].append(1 if get_score(infos[i - 1]) * get_score(infos[i]) > 0 else 0)

        # 8. Близость к мату
        data['mate_distance'].append(-get_mate_distance(infos[i]))

        # 9. Уязвимость короля противника
        board.push(move)
        king_square = board.king(board.turn)
        data['king_under_attack'].append(1 if board.is_attacked_by(not board.turn, king_square) else 0)
        board.pop()

        # 10-12. Тактические угрозы и приёмы
        data['is_capture'].append(board.is_capture(move))
        board.push(move)
        data['is_check'].append(board.is_check())
        data['is_checkmate'].append(board.is_checkmate())
        board.pop()
        data['cnt_promotion'].append(sum(1 for mv in board.legal_moves if mv.promotion) // 4)
        data['is_used_promotion'].append(move.promotion is not None)
        data['is_castling'].append(board.is_castling(move))
    
        # Делаем ход и переходим на следующую итерацию
        board.push(move)
    
    return pd.DataFrame(data)


In [24]:
engine = chess.engine.SimpleEngine.popen_uci(path_to_stockfish)
engine.configure({'Threads': 2})
engine.configure({'Hash': 512})

with open('tmp_data/example_data.pgn', 'r') as pgn_file:
    game = [chess.pgn.read_game(pgn_file) for _ in range(6)][-1]
    print(game.headers['Site'])

data = analyze_game(game, engine, 'lol.csv', analyze_detph=20)

https://lichess.org/zdIVv4Fu


In [25]:
data

,delta_score,deviation_from_pv,threats_to_opponent,threats_from_opponent,legal_moves_diff,sacrifice,advantage_change,mate_distance,king_under_attack,is_capture,is_check,is_checkmate,cnt_promotion,is_used_promotion,is_castling
0,-1.666667,0,0,0,0,0,0,0.000000,0,False,False,False,0,False,False
1,9.666667,0,0,0,-1,0,0,0.000000,0,False,False,False,0,False,False
2,10.000000,0,1,1,1,0,0,0.000000,0,False,False,False,0,False,False
3,1.000000,0,1,1,0,0,0,0.000000,0,False,False,False,0,False,False
4,2.666667,1,1,1,0,0,0,0.000000,0,False,False,False,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1.666667,0,2,2,-23,0,0,2.000000,1,False,True,False,0,False,False
71,1.333333,0,2,1,4,0,0,-3.333333,0,False,False,False,0,False,False
72,2.333333,0,2,2,-27,0,0,1.000000,1,False,True,False,1,False,False
73,1.333333,0,3,2,-1,0,0,-2.333333,0,False,False,False,0,False,False
